In [2]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline

In [6]:
pitch_velo.head()

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,playing_level,pitch_speed_mph,filename_new,modelname_new
0,750,1031,1031_2,110.6784,1.9304,21.40,college,90.4,000750_001031_76_244_013_FF_904.c3d,000750_001031_model.c3d
1,750,1031,1031_3,110.6784,1.9304,21.40,college,90.4,000750_001031_76_244_014_FF_904.c3d,000750_001031_model.c3d
2,774,1097,1097_1,101.1528,1.9304,19.41,college,77.6,000774_001097_76_223_001_FF_776.c3d,000774_001097_model.c3d
3,774,1097,1097_2,101.1528,1.9304,19.41,college,77.0,000774_001097_76_223_005_FF_770.c3d,000774_001097_model.c3d
4,774,1097,1097_3,101.1528,1.9304,19.41,college,76.1,000774_001097_76_223_006_FF_761.c3d,000774_001097_model.c3d


In [8]:
pitch_velo = pd.read_csv('metadata.csv')
force_plate = pd.read_csv('force_plate.csv')
energy_flow = pd.read_csv('energy_flow.csv')


In [14]:
br_fp = force_plate[force_plate['time']== force_plate['BR_time']]
br_ef = energy_flow[energy_flow['time']== energy_flow['BR_time']]
br_ef.head()

,session_pitch,time,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,rear_knee_energy_generated,lead_knee_energy_transfer_stp,lead_knee_energy_transfer_jfp,lead_knee_energy_generated,rear_hip_energy_transfer_stp,rear_hip_energy_transfer_jfp,...,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
482,1031_2,1.3389,-529.6774,1045.7377,-411.0764,0.0,986.7805,1803.8335,0.0000,-707.0322,...,2736.1917,-1791.9441,-5.0074,104.9839,0.6250,1.1806,1.1917,1.3056,1.3389,1.3778
847,1031_3,0.8250,-563.0821,1149.2876,-513.8955,0.0,747.2073,1382.6419,0.0000,-662.1831,...,2458.5579,-1770.0225,10.5157,72.3100,0.0917,0.6639,0.6806,0.7917,0.8250,0.8611
1278,1097_1,1.0028,-50.1856,-85.2700,61.9311,0.0,590.3287,-276.9181,-105.7167,-1053.7740,...,1441.3354,-1680.0410,1159.4906,-135.4557,0.1056,0.7722,0.8278,0.9722,1.0028,1.0500
1730,1097_2,1.0611,274.8673,13.4430,171.8570,0.0,1209.1444,399.2850,0.0000,-1085.4525,...,1077.9598,-3193.7871,49.5306,460.1674,0.1722,0.8528,0.8917,1.0306,1.0611,1.1083
2165,1097_3,1.0306,78.2917,-202.7941,-65.5761,0.0,549.1521,-559.9877,0.0000,-1180.8433,...,1263.8380,-1698.6222,1165.5778,-124.5025,0.1472,0.8361,0.8472,1.0028,1.0306,1.0778


In [15]:
br_fp.head()

,session_pitch,time,rear_force_x,rear_force_y,rear_force_z,lead_force_x,lead_force_y,lead_force_z,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
1446,1031_2,1.3389,0.1670,-0.2389,-0.3254,1164.8713,25.8658,1930.4608,0.6250,1.1806,1.1917,1.3056,1.3389,1.3778
2541,1031_3,0.8250,-0.1859,0.1608,-0.6128,1259.8369,-0.2215,1876.8320,0.0917,0.6639,0.6806,0.7917,0.8250,0.8611
3834,1097_1,1.0028,-0.1578,0.9602,0.2848,554.7148,-141.8671,1386.5601,0.1056,0.7722,0.8278,0.9722,1.0028,1.0500
5190,1097_2,1.0611,-0.1485,0.1819,-0.2126,736.8475,-101.6511,1709.2106,0.1722,0.8528,0.8917,1.0306,1.0611,1.1083
6495,1097_3,1.0306,-0.0905,0.3026,0.1070,636.3936,-70.7661,1470.4899,0.1472,0.8361,0.8472,1.0028,1.0306,1.0778


In [ ]:
df1 = pd.merge(br_ef, br_fp, how='inner', left_on='time', right_on='time')
df = pd.merge(df1, pitch_velo, how='inner', left_on='session_pitch_x', right_on='session_pitch')
df.head()

In [4]:
ball_release.head()
##gets ball release frames from force plate data

,session_pitch,time,rear_force_x,rear_force_y,rear_force_z,lead_force_x,lead_force_y,lead_force_z,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
1446,1031_2,1.3389,0.1670,-0.2389,-0.3254,1164.8713,25.8658,1930.4608,0.6250,1.1806,1.1917,1.3056,1.3389,1.3778
2541,1031_3,0.8250,-0.1859,0.1608,-0.6128,1259.8369,-0.2215,1876.8320,0.0917,0.6639,0.6806,0.7917,0.8250,0.8611
3834,1097_1,1.0028,-0.1578,0.9602,0.2848,554.7148,-141.8671,1386.5601,0.1056,0.7722,0.8278,0.9722,1.0028,1.0500
5190,1097_2,1.0611,-0.1485,0.1819,-0.2126,736.8475,-101.6511,1709.2106,0.1722,0.8528,0.8917,1.0306,1.0611,1.1083
6495,1097_3,1.0306,-0.0905,0.3026,0.1070,636.3936,-70.7661,1470.4899,0.1472,0.8361,0.8472,1.0028,1.0306,1.0778


In [5]:
pitch_velo['mph_bin'] = pd.qcut(pitch_velo['pitch_speed_mph'], q=3, labels = ['bad', 'average', 'elite'])
pitch_velo[:10]

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,playing_level,pitch_speed_mph,mph_bin
0,750,1031,1031_2,110.6784,1.9304,21.40,college,90.4,elite
1,750,1031,1031_3,110.6784,1.9304,21.40,college,90.4,elite
2,774,1097,1097_1,101.1528,1.9304,19.41,college,77.6,bad
3,774,1097,1097_2,101.1528,1.9304,19.41,college,77.0,bad
4,774,1097,1097_3,101.1528,1.9304,19.41,college,76.1,bad
5,653,1170,1170_1,82.1016,1.9304,20.52,college,80.9,bad
6,653,1170,1170_2,82.1016,1.9304,20.52,college,82.6,bad
7,738,1250,1250_1,78.4728,1.7526,21.49,college,88.1,elite
8,738,1250,1250_3,78.4728,1.7526,21.49,college,88.6,elite
9,789,1313,1313_1,84.3696,1.8288,19.02,college,85.8,average


In [6]:
label_bin = LabelEncoder()

In [7]:
pitch_velo['mph_bin'] = label_bin.fit_transform(pitch_velo['mph_bin'])
pitch_velo['mph_bin'].value_counts()

1    138
0    137
2    136
Name: mph_bin, dtype: int64

In [9]:
len(ball_release)
len(pitch_velo)

411

In [11]:
#creating test and training splits to train and test the model
X = df.drop(labels = ['mph_bin', 'pitch_speed_mph', 'user', 'playing_level', 'age_yrs', 'session', 'session_pitch'], axis = 1)
y = df['mph_bin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 12)

In [12]:
#scale values
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
predict_rfc = rfc.predict(X_test)

In [23]:
print(classification_report(y_test, predict_rfc))
print(confusion_matrix(y_test, predict_rfc))
##67, 73, 

              precision    recall  f1-score   support

           0       0.62      0.67      0.64        24
           1       0.73      0.73      0.73        30
           2       0.76      0.70      0.73        27

    accuracy                           0.70        81
   macro avg       0.70      0.70      0.70        81
weighted avg       0.71      0.70      0.70        81

[[16  3  5]
 [ 7 22  1]
 [ 3  5 19]]


In [24]:
clf = svm.SVC()
clf.fit(X_train, y_train)
predict_clf = clf.predict(X_test)

In [25]:
print(classification_report(y_test, predict_clf))
print(confusion_matrix(y_test, predict_clf))
#consistently 50%

              precision    recall  f1-score   support

           0       0.33      0.42      0.37        24
           1       0.61      0.47      0.53        30
           2       0.57      0.59      0.58        27

    accuracy                           0.49        81
   macro avg       0.50      0.49      0.49        81
weighted avg       0.51      0.49      0.50        81

[[10  6  8]
 [12 14  4]
 [ 8  3 16]]
